<a href="https://colab.research.google.com/github/velosomn/datasciencecoursera/blob/master/criptotroxa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

import time

totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls "gdrive/My Drive/Colab Notebooks/Kaggle"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 Antigo			       'Material Sabatina.rar'	 Prova.pdf
'Conversa com Helder.rtf'      'Prova agosto 21'	 provinha.R
'Conversa prova Gustavo.docx'  'Prova agosto 21.rar'	'Results 2.xlsx'
'Cópia de Prova (1).ipynb'      Prova.docx		 Results.xlsx
'Cópia de Prova.ipynb'	       'Prova Fotos (1).ipynb'	 Simulado.ipynb
 Kaggle			       'Prova Fotos.ipynb'	 Templates.ipynb
'Material Sabatina'	        Prova.ipynb		 Untitled


In [11]:
!ls "gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition"

asset_details.csv	       example_test.csv		   treinocompactado.zip
example_sample_submission.csv  supplemental_train.csv.zip


In [12]:
import zipfile
base_treino=0
with zipfile.ZipFile("gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/treinocompactado.zip", 'r') as zip_ref:
  base_treino = pd.read_csv(zip_ref.open('train.csv'))

In [13]:
base_treino["dia_semana"]=pd.to_datetime(base_treino['timestamp'], unit='s').dt.dayofweek
base_treino['hora'] = pd.to_datetime(base_treino['timestamp'], unit='s').dt.hour

In [14]:
base_treino.sample(100)

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,dia_semana,hora
3899038,1537929900,1,130.0,6405.552500,6409.9200,6400.29000,6406.465000,24.838969,6405.875307,0.000528,2,2
16149967,1597368060,11,17.0,92.080000,92.1100,91.99000,92.025000,102.691590,92.038798,0.002339,4,1
8707670,1561739520,7,115.0,8.101180,8.4690,7.92600,8.120460,3342.756214,8.111886,0.008808,4,16
20748271,1617227160,10,24.0,2124.404867,2127.1661,2123.32000,2125.247600,2.526041,2124.497420,0.002203,2,21
5480315,1546027560,11,1.0,49.800000,49.8000,49.80000,49.800000,1.687029,49.800000,NaN,4,20
...,...,...,...,...,...,...,...,...,...,...,...,...
23697386,1629870300,4,360.0,0.292897,0.2932,0.29258,0.292838,942119.708228,0.292802,0.006319,2,5
2268186,1529378520,0,54.0,16.412100,16.4739,16.41200,16.473800,1558.800000,16.462204,0.001564,1,3
8460639,1560594960,1,332.0,8656.303333,8681.8300,8634.20000,8656.096667,18.741554,8656.981482,0.001598,5,10
20225514,1614986700,9,479.0,180.446667,180.7600,180.08000,180.330667,1493.456564,180.322629,-0.001362,4,23


In [15]:
base_treino.shape

(24236806, 12)

# 1. Estruturar uma base por asset

In [16]:
assets = {x: base_treino[base_treino.Asset_ID == x].drop('Asset_ID', axis=1) for x in base_treino.Asset_ID.unique()}

Exemplo :
    assets = {2:base_treino[base_treino.Asset_ID==2],3:base_treino[base_treino.Asset_ID==3]...}

In [17]:
assets[0].head()

,timestamp,Count,Open,High,Low,Close,Volume,VWAP,Target,dia_semana,hora
1,1514764860,5.0,8.5300,8.5300,8.5300,8.5300,78.38,8.530000,-0.014399,0,0
9,1514764920,7.0,8.5300,8.5300,8.5145,8.5145,71.39,8.520215,-0.015875,0,0
17,1514764980,45.0,8.5065,8.5299,8.4848,8.4848,1546.82,8.501394,-0.015410,0,0
25,1514765040,14.0,8.5009,8.5066,8.4744,8.5009,125.80,8.479810,-0.012524,0,0
33,1514765100,5.0,8.5007,8.5007,8.4560,8.4560,125.01,8.458435,-0.005940,0,0


# 2. Tratamento inicial

## 2.1 Timestamp como index

In [18]:
for asset in assets:
    assets[asset] = assets[asset].set_index('timestamp').sort_index()

In [19]:
assets[0].head()

,Count,Open,High,Low,Close,Volume,VWAP,Target,dia_semana,hora
timestamp,,,,,,,,,,
1514764860,5.0,8.5300,8.5300,8.5300,8.5300,78.38,8.530000,-0.014399,0,0
1514764920,7.0,8.5300,8.5300,8.5145,8.5145,71.39,8.520215,-0.015875,0,0
1514764980,45.0,8.5065,8.5299,8.4848,8.4848,1546.82,8.501394,-0.015410,0,0
1514765040,14.0,8.5009,8.5066,8.4744,8.5009,125.80,8.479810,-0.012524,0,0
1514765100,5.0,8.5007,8.5007,8.4560,8.4560,125.01,8.458435,-0.005940,0,0


## 2.2 Tratamento dos missings

In [20]:
# Função para uso futuro

def preenche_missing():
    return 0

In [21]:
for asset in assets:
    assets[asset] = assets[asset].reindex(range(assets[asset].index[0], assets[asset].index[-1] + 60, 60), method='pad').replace([np.inf, -np.inf], np.nan)

In [22]:
assets[0].info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1956960 entries, 1514764860 to 1632182400
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Count       1956960 non-null  float64
 1   Open        1956960 non-null  float64
 2   High        1956960 non-null  float64
 3   Low         1956960 non-null  float64
 4   Close       1956960 non-null  float64
 5   Volume      1956960 non-null  float64
 6   VWAP        1956960 non-null  float64
 7   Target      1929248 non-null  float64
 8   dia_semana  1956960 non-null  int64  
 9   hora        1956960 non-null  int64  
dtypes: float64(8), int64(2)
memory usage: 149.3 MB


In [23]:
for asset in assets:
    print(f'-----{asset}-----')
    print((assets[asset].index[1:] - assets[asset].index[:-1]).value_counts().head())

-----2-----
60    1956959
Name: timestamp, dtype: int64
-----0-----
60    1956959
Name: timestamp, dtype: int64
-----1-----
60    1956959
Name: timestamp, dtype: int64
-----5-----
60    1956959
Name: timestamp, dtype: int64
-----7-----
60    1956959
Name: timestamp, dtype: int64
-----6-----
60    1956959
Name: timestamp, dtype: int64
-----9-----
60    1956959
Name: timestamp, dtype: int64
-----11-----
60    1956959
Name: timestamp, dtype: int64
-----13-----
60    1903823
Name: timestamp, dtype: int64
-----12-----
60    1889287
Name: timestamp, dtype: int64
-----3-----
60    1803769
Name: timestamp, dtype: int64
-----8-----
60    1772153
Name: timestamp, dtype: int64
-----10-----
60    1770279
Name: timestamp, dtype: int64
-----4-----
60    1285046
Name: timestamp, dtype: int64


# 3. Criação de features

## 3.1 Features gerais - vai ter para todos assets

Fazer a construção de features usando funções, pra facilitar a aplicação nos assets

Features já existentes
* timestamp: All timestamps are returned as second Unix timestamps (the number of seconds elapsed since 1970-01-01 00:00:00.000 UTC). Timestamps in this dataset are multiple of 60, indicating minute-by-minute data.
* Asset_ID: The asset ID corresponding to one of the crytocurrencies (e.g. Asset_ID = 1 for Bitcoin). The mapping from Asset_ID to crypto asset is contained in asset_details.csv.
* Count: Total number of trades in the time interval (last minute).
* Open: Opening price of the time interval (in USD).
* High: Highest price reached during time interval (in USD).
* Low: Lowest price reached during time interval (in USD).
* Close: Closing price of the time interval (in USD).
* Volume: Quantity of asset bought or sold, displayed in base currency USD.
* VWAP: The average price of the asset over the time interval, weighted by volume. VWAP is an aggregated form of trade data.
* Target: Residual log-returns for the asset over a 15 minute horizon.

Ideias de features
* Dia da semana
* Feriados Americanos e Chineses
* Volatilidade exponencial - http://ferramentasdoinvestidor.com.br/gestao-de-risco/volatilidade-ewma-no-excel/
* Indice de força relativa
* Médias moveis
* Bandas de bollinger
* MACD
* Agulhada do DIDI - https://www.investimentonabolsa.com/2014/11/saiba-utilizar-agulhada-do-didi.html
* Analise de fourrier - https://journals.sagepub.com/doi/pdf/10.5772/56839
* Nuvens de ichimoku - https://www.investopedia.com/terms/i/ichimoku-cloud.asp
* numero de transações de bitcoin - observar interesse do publico em cripto

In [24]:
#def media_movel_30(precos):
#    return precos.rolling(30).mean()

def media_movel(precos,intervalo):
    return precos.rolling(intervalo).mean()

In [25]:
def get_bollinger_bands(dfbol, rate=30):
    sma = np.array(dfbol.rolling(rate).mean())
    std = np.array(dfbol.rolling(rate).std())
    boll_up =  sma + std * 2 # Calculate top band
    boll_down = sma - std * 2 # Calculate bottom band
    boll=np.zeros(dfbol.size)
    preco=np.array(dfbol)
    indexes = list(range(dfbol.size))
    for i in indexes:
        if boll_up[i]<preco[i]:
            boll[i]=1
        elif boll_down[i]>preco[i]:
            boll[i]=-1
        else:
            boll[i]=0
    return boll

In [26]:
def agulhada_didi(dfagu):
    didi3 = np.array(dfagu.rolling(3).mean())
    didi8 = np.array(dfagu.rolling(8).mean())
    didi20 = np.array(dfagu.rolling(20).mean())
    Didi=np.zeros(dfagu.size)
    indexes = list(range(dfagu.size))
    for i in indexes:
        if ((didi3[i]>didi8[i]) & (didi8[i]>didi20[i])):
            Didi[i]=1
        else:
            Didi[i]=0
    return Didi

In [27]:
    def MACD(dfmacd):
        exp1 = dfmacd.ewm(span=12, adjust=False).mean()
        exp2 = dfmacd.ewm(span=26, adjust=False).mean()
        diff = exp1-exp2
        diff_np=np.array(diff)
        exp3 = np.array(diff.ewm(span=9, adjust=False).mean())
        MACD=np.zeros(dfmacd.size)
        indexes = list(range(dfmacd.size))
        for i in indexes:
            if (diff_np[i]>=exp3[i]):
                MACD[i]=1
            else:
                MACD[i]=-1        
        return MACD

In [28]:
def willR (dfw,dfhigh,dflow,bins):
    high = [np.array(dfhigh.rolling(bins).max(),dtype=float)-np.array(dfw,dtype=float)]
    low = [np.array(dfhigh.rolling(bins).max(),dtype=float)-np.array(dflow.rolling(bins).min(),dtype=float)]
    #
    willr = np.transpose(np.nan_to_num(np.true_divide(high,low,out=np.zeros_like(high), where=low!=0)*(-100)))
    
    willr_flag=np.zeros(dfw.size)
    indexes = list(range(dfw.size))
    for i in indexes:
        if (willr[i]>-20):
            willr_flag[i]=1
        elif (willr[i]<-80):
            willr_flag[i]=-1
        else:
            willr_flag[i]=0

    return willr,willr_flag

In [29]:
def log_return(series, periods=1):
    return np.log(series).diff(periods=periods)

In [30]:
for base in assets:
    
    #simple moving average
    assets[base]['SMA_30'] = media_movel(assets[base]['Close'],30)
    assets[base]['SMA_15'] = media_movel(assets[base]['Close'],15)
    
    #log simple moving average
    assets[base]['log_SMA_30'] = np.log(assets[base]['Close'].rolling(30).mean())
    assets[base]['log_SMA_15'] = np.log(assets[base]['Close'].rolling(15).mean())
    
    #log cumulative moving average
    assets[base]['log_CMA_30'] = np.log(assets[base]['Close'].expanding(30).mean())
    assets[base]['log_CMA_15'] = np.log(assets[base]['Close'].expanding(15).mean())
    
    #log exponential moving average
    assets[base]['log_EWM_30'] = np.log(assets[base]['Close'].ewm(span=30).mean())
    assets[base]['log_EWM_15'] = np.log(assets[base]['Close'].ewm(span=15).mean())

    #Bollinger
    assets[base]['boll_30'] = get_bollinger_bands(assets[base]['Close'],30)
    #DIDI Index
    assets[base]['Didi'] = agulhada_didi(assets[base]['Close'])
    #MACD
    assets[base]['MACD'] = MACD(assets[base]['Close'])
    #Williams R
    assets[base]['willr'],assets[base]['willr_flag'] = willR(assets[base]['Close'],assets[base]['High'],assets[base]['Low'],14)
    #Log do Retorno
    assets[base]["log_ret"]=log_return(assets[base]['Close'])[1:]
 
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [31]:
assets[0].sample(100)

,Count,Open,High,Low,Close,Volume,VWAP,Target,dia_semana,hora,SMA_30,SMA_15,log_SMA_30,log_SMA_15,log_CMA_30,log_CMA_15,log_EWM_30,log_EWM_15,boll_30,Didi,MACD,willr,willr_flag,log_ret
timestamp,,,,,,,,,,,,,,,,,,,,,,,,
1569379860,42.0,16.22250,16.3030,16.1246,16.23885,1268.9500,16.228687,-0.000173,2,2,16.233178,16.217593,2.787057,2.786097,2.731791,2.731791,2.787741,2.786209,0.0,0.0,1.0,-51.258532,0.0,0.000755
1583659860,37.0,19.47090,19.4709,19.4507,19.45360,402.2300,19.457544,0.000242,6,9,19.426897,19.451343,2.966659,2.967916,2.762674,2.762674,2.966768,2.967784,0.0,1.0,-1.0,-46.725860,0.0,-0.000915
1624575300,322.0,308.62350,309.4100,308.4900,309.31000,677.4299,308.929520,-0.003561,3,22,309.046733,309.043167,5.733493,5.733481,4.026569,4.026569,5.733422,5.732818,0.0,0.0,-1.0,-26.190476,0.0,0.002541
1556808120,128.0,23.95070,23.9570,23.9096,23.91710,3039.9000,23.937386,-0.010221,3,14,23.796453,23.908387,3.169537,3.174229,2.444891,2.444891,3.169161,3.173970,0.0,0.0,-1.0,-50.335731,0.0,-0.001383
1551484320,168.0,11.30770,11.3131,11.2662,11.29380,2885.1900,11.284906,-0.002479,4,23,11.356967,11.362953,2.429831,2.430358,2.362636,2.362636,2.430170,2.428957,-1.0,0.0,-1.0,-79.615953,0.0,-0.000513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1624849440,229.0,287.59800,287.8000,287.2100,287.71800,577.9752,287.628890,-0.003275,0,3,287.306650,287.181167,5.660550,5.660113,4.036609,4.036609,5.660586,5.660624,0.0,1.0,1.0,-20.760870,0.0,0.001362
1598284020,41.0,22.51480,22.5335,22.5136,22.52750,929.3400,22.528961,-0.000877,0,15,22.479713,22.490717,3.112613,3.113103,2.772557,2.772557,3.113158,3.113303,1.0,1.0,1.0,-6.904488,1.0,0.000235
1561690860,96.0,34.15615,34.2554,34.0909,34.20710,1519.0710,34.167009,0.004165,4,3,34.448128,34.393477,3.539455,3.537867,2.600602,2.600602,3.538496,3.536608,0.0,0.0,-1.0,-79.681763,0.0,0.001092


## 3.2 Features específicas (opcionais)

### 3.2.1 BTC

### 3.2.1 ETH

# 4. Modelagem

## 4.1 Quebra de treino/validação/teste

In [32]:
list_periodos = []

for asset in base_treino.Asset_ID.unique():
    maximo = base_treino[base_treino.Asset_ID == asset].timestamp.max()
    minimo = base_treino[base_treino.Asset_ID == asset].timestamp.min()
    list_periodos.append([asset,minimo,maximo])
    
df_periodos = pd.DataFrame(list_periodos, columns=['asset','min','max'])
df_periodos['min'] = pd.to_datetime(df_periodos['min'], unit='s')
df_periodos['max'] = pd.to_datetime(df_periodos['max'], unit='s')
df_periodos.sort_values('min')

,asset,min,max
0,2,2018-01-01 00:01:00,2021-09-21
1,0,2018-01-01 00:01:00,2021-09-21
2,1,2018-01-01 00:01:00,2021-09-21
3,5,2018-01-01 00:01:00,2021-09-21
4,7,2018-01-01 00:01:00,2021-09-21
5,6,2018-01-01 00:01:00,2021-09-21
6,9,2018-01-01 00:01:00,2021-09-21
7,11,2018-01-01 00:01:00,2021-09-21
8,13,2018-02-06 21:37:00,2021-09-21
9,12,2018-02-16 23:53:00,2021-09-21


In [ ]:
from sklearn.preprocessing import StandardScaler

assets_scalers = {
    asset: StandardScaler() for asset in assets
}

In [ ]:
from tqdm import tqdm

DATA_CORTE = "21/08/2021"
DATA_CORTE_MIN = "13/04/2019"


# train_windows = {
#     asset: [totimestamp(df_periodos.loc[df_periodos['asset'] == asset, 'min'].dt.strftime('%d/%m/%Y').item()), totimestamp(DATA_CORTE)] for asset in assets
# }

train_windows = {
    asset: [totimestamp(DATA_CORTE_MIN), totimestamp(DATA_CORTE)] for asset in assets
}

test_window = [totimestamp(DATA_CORTE), totimestamp("21/09/2021")]

assets_train = {
    asset: {
        'features': assets_scalers[asset].fit_transform(
            assets[asset].drop('Target', axis=1).loc[train_windows[asset][0]:train_windows[asset][1]].fillna(preenche_missing())
            ),
        'target': assets[asset].loc[train_windows[asset][0]:train_windows[asset][1]]['Target'].fillna(preenche_missing())} for asset in tqdm(assets)
}

assets_test = {
    asset: {
        'features': assets_scalers[asset].transform(
            assets[asset].drop('Target', axis=1).loc[test_window[0]:test_window[1]].fillna(preenche_missing())
            ),
        'target': assets[asset].loc[test_window[0]:test_window[1]]['Target'].fillna(preenche_missing())} for asset in tqdm(assets)
}

NameError: ignored

In [ ]:
assets_train[8].shape

## 4.2 Modelo baseline

In [ ]:
class BaselineModel:
    def __init__(self):
        pass
    
    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train

    
    def predict(self, x):
        

## 4.3 Modelagem freestyle

In [33]:
##4.4 Modelagem Marcelo

In [ ]:
def get_Xy_and_model_for_asset(df_proc):
    from lightgbm import LGBMRegressor
    df_proc = df_proc.replace([np.inf, -np.inf], np.nan).dropna(how="any")
    
    X = np.array(df_proc.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'VWAP','Target'], axis=1))
    y = np.array(df_proc["Target"])

    model = LGBMRegressor( 
                          n_estimators = 800, 
                          learning_rate = 0.01, 
                          max_bin = 15, 
                          num_leaves = 100
                        
                          )
    model.fit(X, y)
    return X, y, model

In [ ]:
Xs = {}
ys = {}
models = {}

In [ ]:
X, y, model = get_Xy_and_model_for_asset(assets[i])

In [ ]:
Xs = {}
ys = {}
models = {}
for asset_id in list(list_id):
    X, y, model = get_Xy_and_model_for_asset(assets[asset_id])
    Xs[asset_id], ys[asset_id], models[asset_id] = X, y, model
print("Models training finished")

In [ ]:
fi_df = pd.DataFrame()
fi_df['features'] = assets[1].drop(['Open', 'High', 'Low', 'Close', 'Volume', 'VWAP','Target'],axis=1).columns
fi_df['importance'] = models[1].booster_.feature_importance(importance_type="gain")

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
print(f'MAE Of model : {mean_absolute_error(ys[0], models[0].predict(Xs[0])):.6f}')
print(f'MSE Of model : {mean_squared_error(ys[0], models[0].predict(Xs[0])):.6f}')
print('*'*15,'The evaluation done','*'*15) 

In [ ]:
# plot feature importance
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(1, 1, figsize=(7, 15))
sns.barplot(
    x='importance'
    , y='features'
    , data=fi_df.sort_values(by=['importance'], ascending=False)
    , ax=ax
)

# 5. Avaliação dos resultados